In [77]:
import pandas as pd
import pint

In [88]:

units = pint.UnitRegistry()


In [89]:
DATA = pd.read_csv('../DATA/data.csv')
L_1, L_2 = units.Quantity(58.2, 'cm'), units.Quantity(48.2, 'cm')
L_i = [L_1, L_2]
for column , l in zip(DATA.columns[1:], L_i):
    print(f"{column} == 'm': {column == 'm'}")
    DATA[column] = DATA[column].apply(lambda dx: dx*units.millimeter)
    DATA[f"sin_phi_{column}"] = DATA[column].apply(lambda dx: dx/(l.to(units.millimeter)))

DATA

Delta_L_1_mm == 'm': False
Delta_L_2_mm == 'm': False


,m,Delta_L_1_mm,Delta_L_2_mm,sin_phi_Delta_L_1_mm,sin_phi_Delta_L_2_mm
0,1,-1.32 millimeter,1.1 millimeter,-0.002268041237113402 dimensionless,0.002282157676348548 dimensionless
1,2,-5.09 millimeter,3.47 millimeter,-0.008745704467353952 dimensionless,0.007199170124481328 dimensionless
2,3,-6.62 millimeter,5.21 millimeter,-0.011374570446735396 dimensionless,0.010809128630705394 dimensionless


In [90]:
d = lambda m, lamb, sin_phi: (m + 0.5) * (lamb / sin_phi)
lambda_ = 632.8 * units.nanometer

for i in range(1, 3):
    DATA[f"d_{i}"] = DATA.apply(
        lambda row: d(
            row['m'],  # Use the 'm' column for the integer value
            lambda_,
            row[f'sin_phi_Delta_L_{i}_mm']
        ),
        axis=1
    )


In [91]:
DATA

,m,Delta_L_1_mm,Delta_L_2_mm,sin_phi_Delta_L_1_mm,sin_phi_Delta_L_2_mm,d_1,d_2
0,1,-1.32 millimeter,1.1 millimeter,-0.002268041237113402 dimensionless,0.002282157676348548 dimensionless,-418510.90909090906 nanometer,415922.18181818177 nanometer
1,2,-5.09 millimeter,3.47 millimeter,-0.008745704467353952 dimensionless,0.007199170124481328 dimensionless,-180888.80157170922 nanometer,219747.55043227662 nanometer
2,3,-6.62 millimeter,5.21 millimeter,-0.011374570446735396 dimensionless,0.010809128630705394 dimensionless,-194715.0453172205 nanometer,204900.8829174664 nanometer


In [92]:
d_1_avg = DATA['d_1'].apply(lambda x: x.to(units.millimeter).magnitude).mean() * units.millimeter
d_2_avg = DATA['d_2'].apply(lambda x: x.to(units.millimeter).magnitude).mean() * units.millimeter

In [93]:
# adding one more row to data frame labeled 'average'
average_row = pd.DataFrame([{
    'm': 'average',
    'Delta_L_1_mm': DATA['Delta_L_1_mm'].apply(lambda x: x.magnitude).mean() * units.millimeter,
    'Delta_L_2_mm': DATA['Delta_L_2_mm'].apply(lambda x: x.magnitude).mean() * units.millimeter,
    'sin_phi_Delta_L_1_mm': DATA['sin_phi_Delta_L_1_mm'].apply(lambda x: x.magnitude).mean(),
    'sin_phi_Delta_L_2_mm': DATA['sin_phi_Delta_L_2_mm'].apply(lambda x: x.magnitude).mean(),
    'd_1': d_1_avg,  # Convert to magnitude
    'd_2': d_2_avg   # Convert to magnitude
}])


DATA = pd.concat([DATA, average_row], ignore_index=True)


In [94]:
DATA

,m,Delta_L_1_mm,Delta_L_2_mm,sin_phi_Delta_L_1_mm,sin_phi_Delta_L_2_mm,d_1,d_2
0,1,-1.32 millimeter,1.1 millimeter,-0.002268041237113402 dimensionless,0.002282157676348548 dimensionless,-418510.90909090906 nanometer,415922.18181818177 nanometer
1,2,-5.09 millimeter,3.47 millimeter,-0.008745704467353952 dimensionless,0.007199170124481328 dimensionless,-180888.80157170922 nanometer,219747.55043227662 nanometer
2,3,-6.62 millimeter,5.21 millimeter,-0.011374570446735396 dimensionless,0.010809128630705394 dimensionless,-194715.0453172205 nanometer,204900.8829174664 nanometer
3,average,-4.343333333333334 millimeter,3.2600000000000002 millimeter,-0.007463,0.006763,-0.26470491865994633 millimeter,0.280190205055975 millimeter


In [95]:
d_1_avg, d_2_avg = d_1_avg.to(units.millimeter), d_2_avg.to(units.millimeter)
d_1_avg, d_2_avg

(<Quantity(-0.26470491865994633, 'millimeter')>,
 <Quantity(0.280190205055975, 'millimeter')>)

In [96]:
DATA['d_1'] = DATA['d_1'].apply(lambda x: x.to(units.millimeter))
DATA['d_2'] = DATA['d_2'].apply(lambda x: x.to(units.millimeter))



In [97]:
DATA

,m,Delta_L_1_mm,Delta_L_2_mm,sin_phi_Delta_L_1_mm,sin_phi_Delta_L_2_mm,d_1,d_2
0,1,-1.32 millimeter,1.1 millimeter,-0.002268041237113402 dimensionless,0.002282157676348548 dimensionless,-0.41851090909090916 millimeter,0.41592218181818186 millimeter
1,2,-5.09 millimeter,3.47 millimeter,-0.008745704467353952 dimensionless,0.007199170124481328 dimensionless,-0.18088880157170925 millimeter,0.21974755043227664 millimeter
2,3,-6.62 millimeter,5.21 millimeter,-0.011374570446735396 dimensionless,0.010809128630705394 dimensionless,-0.19471504531722053 millimeter,0.20490088291746644 millimeter
3,average,-4.343333333333334 millimeter,3.2600000000000002 millimeter,-0.007463,0.006763,-0.26470491865994633 millimeter,0.280190205055975 millimeter


In [101]:
n = 5
for column in DATA.columns[1:]:
    DATA[column] = DATA[column].apply(lambda x: format(x.magnitude, f'.{n}f') if isinstance(x, units.Quantity) else format((float(x)), f'.{n}f'))


In [103]:
DATA

,m,Delta_L_1_mm,Delta_L_2_mm,sin_phi_Delta_L_1_mm,sin_phi_Delta_L_2_mm,d_1,d_2
0,1,-1.32000,1.10000,-0.00227,0.00228,-0.41851,0.41592
1,2,-5.09000,3.47000,-0.00875,0.00720,-0.18089,0.21975
2,3,-6.62000,5.21000,-0.01137,0.01081,-0.19472,0.20490
3,average,-4.34333,3.26000,-0.00746,0.00676,-0.26470,0.28019


In [102]:
DATA.to_csv('../DATA/processed_data.csv', index=False)